In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

torch.random.manual_seed(1)

n_classes = 4 #number of classification catetory

In [2]:
# Make transforms and use data loaders

mean_nums = [0.485, 0.456, 0.406] #use these two as fixed number
std_nums = [0.229, 0.224, 0.225]

data_transforms = transforms.Compose([   #apply these 4 preprcoessing steps
        transforms.Resize(256), #resizing image
        transforms.CenterCrop(224), #crop image to 224*224
        transforms.ToTensor(), #covert to tensor 
        transforms.Normalize(mean_nums, std_nums) # refers to RGB values; 
])

In [3]:
# Set the directory for the data
data_dir = 'Data/Bearing_Images'

# Use the image folder function to create datasets; use sub-folder namers to annotate the images within subfolders
train_data = datasets.ImageFolder(data_dir,transform=data_transforms)

# percentage of training set to use as validation
valid_size = 0.3

# obtain training indices that will be used for validation
num_train = len(train_data) #get total number of data: 240
indices = list(range(num_train)) #create a index: 0-239
np.random.shuffle(indices) #shuffle index
valid_split = int(np.floor((valid_size) * num_train)) #generate a breakpoint to split training and testing: 72
valid_idx, train_idx = indices[:valid_split], indices[valid_split:] # first 72 samples are testing samples; remaining are training samples

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx) #convert list index to be recognizable by torch
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler), batch_size refers to batch training, how many samples to be trained at one iteration
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20,
    sampler=train_sampler) #train_loader contain not only the images but also the labels
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=20, 
    sampler=valid_sampler)

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, padding =1, stride=4), #(224-11+2)/4+1 =54 (20,3,224,224)->(20,96,54,54)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)) #54-3/2 +1=26           (20,96,26,26)
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1), #26-5/1 +1=22 (20,256,22,22)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)) #22-3/2 +1=10 (20,256,10,10)
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, padding =1, stride=1), #10+2-3/1 +1=10 (20,384,10,10)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)) #10-3/2 +1=4  (20,384,4,4)

        self.drop_out = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(6144, 100) #(20,384,4,4) -> (20, 6144) -> (20,100)
        self.fc2 = nn.Linear(100, 4)
        self.softmax = nn.LogSoftmax(dim=1) #if you remove this line, use CrossEntropyLoss
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1) #-1 means the system will do an automatic calculation of 384*4*4
        out = self.fc1(out)
        out = self.drop_out(out)
        out = self.fc2(out)
        out = self.softmax(out) #if you remove this line, use CrossEntropyLoss
        return out

In [5]:
model = ConvNet()
epochs = 20 #difference between epoch and iteration: if don't apply batch training, they are the same; otherwise, one iteration correspond to training one batch of data, and one epoch correspond to training all data batches once.
    
# Loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())
    
#valid_loss_min = np.Inf
    
for epoch in range(epochs):
    
    #model.train()
    train_loss = 0.0
    valid_loss = 0.0
    for inputs, labels in train_loader: #training
        logps = model(inputs)
        loss = criterion(logps, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()    
       
#     model.eval()
#     with torch.no_grad():
    accuracy = 0
    for inputs, labels in valid_loader:
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)
        valid_loss += batch_loss.item()
        # Calculate classification accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
        # calculate average losses
    train_loss = train_loss/len(train_loader)
    valid_loss = valid_loss/len(valid_loader)
    valid_accuracy = accuracy/len(valid_loader) 
      
        # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.6f}'.format(
                epoch + 1, train_loss, valid_loss, valid_accuracy))


Epoch: 1 	Training Loss: 1.619236 	Validation Loss: 0.706025 	Validation Accuracy: 0.591667
Epoch: 2 	Training Loss: 0.563844 	Validation Loss: 0.434780 	Validation Accuracy: 0.783333
Epoch: 3 	Training Loss: 0.446282 	Validation Loss: 0.449098 	Validation Accuracy: 0.670833
Epoch: 4 	Training Loss: 0.388446 	Validation Loss: 0.384137 	Validation Accuracy: 0.720833
Epoch: 5 	Training Loss: 0.422829 	Validation Loss: 0.436953 	Validation Accuracy: 0.708333
Epoch: 6 	Training Loss: 0.386051 	Validation Loss: 0.356606 	Validation Accuracy: 0.791667
Epoch: 7 	Training Loss: 0.372384 	Validation Loss: 0.361097 	Validation Accuracy: 0.750000
Epoch: 8 	Training Loss: 0.359063 	Validation Loss: 0.380679 	Validation Accuracy: 0.695833
Epoch: 9 	Training Loss: 0.279699 	Validation Loss: 0.265257 	Validation Accuracy: 0.804167
Epoch: 10 	Training Loss: 0.461528 	Validation Loss: 1.041569 	Validation Accuracy: 0.929167
Epoch: 11 	Training Loss: 0.916461 	Validation Loss: 0.169351 	Validation Accur

In [6]:
torch.save(model.state_dict(), '2DCNN')